In [1]:
%pylab inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier

from sklearn import model_selection, datasets, metrics
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.feature_extraction import DictVectorizer as DV

from sklearn.tree import export_graphviz
from sklearn.utils import shuffle

from sklearn.feature_selection import RFE

Populating the interactive namespace from numpy and matplotlib


In [2]:
%matplotlib inline

In [3]:
data = pd.read_csv('./kaggle/orange_small_churn_train_data.csv', sep = ',', header = 0, index_col=0)
test_data = pd.read_csv('./kaggle/orange_small_churn_test_data.csv', sep = ',', header = 0, index_col=0)

## Подготовка данных

In [202]:
# т.к. в последней строке тренировочных данных ответа нету, то удаляем её
train_data = data.iloc[:-1,:-1]
train_labels = data.iloc[:-1,-1:]

print train_data.shape
print train_labels.shape
print test_data.shape

(18298, 230)
(18298, 1)
(10000, 230)


In [203]:
# отделяем числовые и категориальные признаки и удаляем полностью пустые признаки
data_numb = train_data.iloc[:,0:190].dropna(axis=1, how='all')
data_categ = train_data.iloc[:,190:].dropna(axis=1, how='all')

In [204]:
columns_name = data_numb.columns.to_list()

In [205]:
# заменим NONE на среднее значение колонки
# Посчитаем средние по колонкам
numeric_means = data_numb.mean(axis=0, skipna=True)

# Заполним пропущенные численные значения средними
data_numb = data_numb.fillna(numeric_means, axis=0)

In [206]:
# попробуем обучить только вещественные признаки чтобы понять какие признакие самые важные
select = RFE(GradientBoostingClassifier(n_estimators = 95, random_state=0, learning_rate=0.09, max_depth=2),
            n_features_to_select=10)

In [129]:
%%time
select.fit(data_numb, train_labels)

KeyboardInterrupt: 

In [ ]:
select.support_

In [207]:
support = array([False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False,  True,
        True, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False])

In [208]:
columns_top10=[columns_name[ind] for ind, col in enumerate(support) if col]

In [209]:
data_numb = data_numb[columns_top10]

In [210]:
# ВЕЩЕСТВЕННЫЕ признаки
# NOTE: сделаем скелинг численных признаков с D=1 (подробнее см. неделю 1)
# NOTE2: такая запись по созданию нового DataFrame нужна из-за тоог что при fit_transform слетают индексы и потом
# при контатенации у нас получается каша.
scaler = StandardScaler()
data_numb=pd.DataFrame(scaler.fit_transform(data_numb.values), index=data_numb.index, columns=columns_top10)

# меняем пустые на 0 (0 это среднее так как признаки масштабированы)
# data_numb.fillna(0, inplace=True)


# КАТЕГОРИАЛЬНЫЕ признаки
# заменим пустые значения на NA (будет как доп.признак)
data_categ = data_categ.fillna('NA').applymap(lambda s: str(s))

# удалим те колонки где больше 20 категорий и меньше 2 (эти колонки не информативны)
name_del = [name for name, var in data_categ.iteritems() if var.value_counts(dropna=True).shape[0] > 20 or var.value_counts(dropna=True).shape[0] < 2]
data_categ = data_categ.drop(labels=name_del, axis=1)

In [211]:
# ВАРИАНТ 1 кодируем категориальные признаки
# data_dummies = pd.get_dummies(data_categ)


# ВАРИАНТ 2 one-hote-encoder
encoder = DV(sparse = False)
data_dummies = encoder.fit_transform(data_categ.T.to_dict().values())

In [212]:
data_dummies.shape

(18298, 101)

In [213]:
# при ВАРИАНТЕ 1
# объединяем числовые признаки и закодированные категориальные
# train_data = pd.concat([data_numb, data_dummies], axis=1)
# train_data.fillna(0, inplace=True)


# при ВАРИАНТЕ 2
train_data = np.hstack((data_numb.values, data_dummies))
# train_data.fillna(0, inplace=True)

In [214]:
print data_numb.shape
print data_categ.shape

print train_data.shape
print train_labels.shape

(18298, 10)
(18298, 20)
(18298, 111)
(18298, 1)


## Балансировка классов

In [215]:
# 1 соответствует классу отток, -1 - классу не отток
# как видим у нас оч большая расбалансировка классов, попробуем её уменьшить
print train_labels["labels"].value_counts()/train_labels.shape[0]

-1.0    0.924746
 1.0    0.075254
Name: labels, dtype: float64


In [216]:
# Попробуем отбалансировать классы, сгенеририрем отдельно индексы для вещественных и категориальных классов
# чтобы у нас получились как будто новые непохожие элементы.

cnt_el = 15000

# получаем только класс 1
only_class_1_numb = data_numb[train_labels['labels']==1]
only_class_1_categ = pd.DataFrame(data_dummies[train_labels['labels']==1])

# сгенерировали индексы значения которых будем дублировать (добавим 15к новых элементов класса 1)
ind_class_1_numb_to_add = np.random.randint(0, only_class_1_numb.shape[0]-1, size=cnt_el)
ind_class_1_categ_to_add = np.random.randint(0, only_class_1_categ.shape[0]-1, size=cnt_el)

# сопоставляем индексы и значения
X_train_to_add_numb = only_class_1_numb.iloc[ind_class_1_numb_to_add]
X_train_to_add_categ = only_class_1_categ.iloc[ind_class_1_categ_to_add]

new_data_class_1 = np.hstack((X_train_to_add_numb.values, X_train_to_add_categ))
new_labels_class_1 = pd.DataFrame(np.ones(cnt_el))

In [217]:
print new_data_class_1.shape
print new_labels_class_1.shape
print train_labels.shape

(15000, 111)
(15000, 1)
(18298, 1)


In [218]:
print type(new_labels_class_1)
print type(train_labels.values)

# new_labels_class_1 = np.array([new_labels_class_1])

<class 'pandas.core.frame.DataFrame'>
<type 'numpy.ndarray'>


In [219]:
train_data = np.vstack((train_data, new_data_class_1))
train_labels = np.vstack((train_labels, new_labels_class_1))

In [220]:
print train_data.shape
print train_labels.shape

(33298, 111)
(33298, 1)


In [221]:
# посмотрим на баланс классов
print len(train_labels[train_labels==1])
print len(train_labels[train_labels==-1])

16377
16921


In [222]:
# перемешаем данные
new_train_data = shuffle(np.hstack((train_data, train_labels)))

In [223]:
type(new_train_data)

numpy.ndarray

In [225]:
train_data = new_train_data[:,:-1]
train_labels = new_train_data[:,-1:]

In [226]:
print train_data.shape
print train_labels.shape

(33298, 111)
(33298, 1)


## Градиентный бустинг деревьев

In [175]:
# max_depth - максимальная глубина
# learning_rate - насколько сильно каждое дерево будет пытаться исправить ошибки предыдущих деревьев.
parameters_grid = {
    'n_estimators' : range(90, 110, 5),
    'learning_rate' : [0.09, 0.1, 0.11],
    'max_depth': range(1, 4, 1)
}

# Будем использовать метод стратификации который делит соотношение классов в обучающей выборке на равное количество
skf = model_selection.StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
classifier = GradientBoostingClassifier(random_state=0)
grid_rfc = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'roc_auc', cv = skf)

In [176]:
%%time
grid_rfc.fit(train_data, train_labels)

CPU times: user 34min 39s, sys: 2.01 s, total: 34min 41s
Wall time: 34min 41s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
       error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [90, 95, 100, 105], 'learning_rate': [0.09, 0.1, 0.11], 'max_depth': [1, 2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [177]:
grid_rfc.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.11, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=105,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [178]:
print grid_rfc.best_score_
print grid_rfc.best_params_

0.8857861846433008
{'n_estimators': 105, 'learning_rate': 0.11, 'max_depth': 3}


-----

In [227]:
# строим модель с оптимальными параметрами которые удалось подобрать
clf = GradientBoostingClassifier(n_estimators = 105, random_state=0, learning_rate=0.11, max_depth=3)
clf.fit(train_data, train_labels)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.11, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=105,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [228]:
# проверяем метрики на тренировочном наборе
actual_labels = clf.predict(train_data)
# actual_labels_proba = clf.predict_proba(train_data)

In [229]:
print "AUC_ROC =", metrics.roc_auc_score(train_labels, actual_labels)
print "accuracy =", clf.score(train_data, train_labels)
print "precision =", metrics.precision_score(train_labels, actual_labels)
print "recall =", metrics.recall_score(train_labels, actual_labels)
print "f1 =", metrics.f1_score(train_labels, actual_labels)

AUC_ROC = 0.7978940001696118
accuracy = 0.7977055679019761
precision = 0.7857269871376918
recall = 0.8094278561397081
f1 = 0.7974013474494707


In [230]:
# AUC_ROC = 0.5021491001564492
# accuracy = 0.9250191277735271
# precision = 0.8571428571428571
# recall = 0.004357298474945534

In [231]:
# важность признаков
clf.feature_importances_

array([3.07890705e-02, 9.07238578e-04, 3.55577003e-03, 1.26684888e-01,
       3.56854869e-02, 2.86772622e-02, 2.72459805e-02, 3.31044695e-01,
       0.00000000e+00, 5.82658231e-02, 1.49232282e-03, 2.22988270e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       9.38748114e-04, 0.00000000e+00, 0.00000000e+00, 5.14969606e-04,
       0.00000000e+00, 1.16677026e-04, 0.00000000e+00, 9.12194414e-04,
       0.00000000e+00, 2.62050730e-04, 1.16055334e-03, 4.38800320e-06,
       0.00000000e+00, 1.48002268e-04, 2.73007002e-02, 1.21889575e-04,
       0.00000000e+00, 0.00000000e+00, 1.79542756e-04, 1.20199733e-02,
       6.12160508e-03, 0.00000000e+00, 0.00000000e+00, 1.22121005e-04,
       9.21120249e-04, 4.88461385e-03, 0.00000000e+00, 1.89963269e-04,
       0.00000000e+00, 0.00000000e+00, 8.94228630e-04, 0.00000000e+00,
       3.19356894e-04, 3.03460451e-03, 1.65690158e-03, 5.77443236e-03,
       3.50571872e-02, 4.69129032e-02, 2.14341434e-03, 1.91557724e-03,
      

## Тестовый набор

In [232]:
# Предобрабатываем тестовый набор
print test_data.shape

# отделяем числовые и категориальные признаки и удаляем полностью пустые признаки
# data_numb_test = test_data.iloc[:,0:190].dropna(axis=1, how='all')
data_numb_test = test_data[columns_top10]
data_categ_test = test_data.iloc[:,190:]

# data_dummies = pd.get_dummies(data_categ)


(10000, 230)


In [233]:
# ВЕЩЕСТВЕННЫЕ признаки
numeric_means_test = data_numb_test.mean(axis=0, skipna=True)
# Заполним пропущенные численные значения средними
data_numb_test = data_numb_test.fillna(numeric_means_test, axis=0)

data_numb_test =pd.DataFrame(scaler.transform(data_numb_test.values), index=data_numb_test.index, columns=columns_top10)

# data_numb_test.fillna(0, inplace=True)

# КАТЕГОРИАЛЬНЫЕ ПРИЗНАКИ
# берем только колонки используемые в обучении
data_categ_test = data_categ_test[data_categ.columns]
data_categ_test = data_categ_test.fillna('NA').applymap(lambda s: str(s))

# удалим те колонки где больше 20 категорий и меньше 2 (эти колонки не информативны)
# name_del = [name for name, var in data_categ_test.iteritems() if var.value_counts(dropna=True).shape[0] > 20 or var.value_counts(dropna=True).shape[0] < 2]
# data_categ_test = data_categ_test.drop(labels=name_del, axis=1)

In [234]:
print data_numb_test.shape
print data_categ_test.shape

(10000, 10)
(10000, 20)


In [235]:
# кодируем категориальные признаки
# data_dummies_test = pd.get_dummies(data_categ_test)

data_dummies_test = encoder.transform(data_categ_test.T.to_dict().values())
# data_dummies_test = np.where(data_dummies_test == np.nan, data_dummies_test, 0)

In [236]:
# объединяем числовые признаки и закодированные категориальные
test_data = np.hstack((data_numb_test.values, data_dummies_test))

In [237]:
print test_data.shape

(10000, 111)


In [238]:
data_dummies_test[:1]

array([[1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0.]])

In [239]:
test_labels = clf.predict(test_data)
test_labels_proba = clf.predict_proba(test_data)

## Сохраняем результат для Kaggle

In [240]:
df = pd.DataFrame(test_labels_proba[:,1], columns=['result'])  
df.index.name = 'ID'
# df['index'] = df.index
# df.astype({"ID": int, "result": float})

In [241]:
# np.savetxt('output.csv', df, delimiter=',', fmt='%f', header='ID, result')
df.to_csv('output.csv', index=True)